### Counting with Cypher Query

In this notebook, we will discuss how to search documents based on the Disease described in the CVD tree.

In [1]:
import pandas as pd
import json
from neo4j import GraphDatabase
import csv

#### Authentication to access covidgraph.org graph

In [2]:
covid_browser = "https://db.covidgraph.org/browser/"
covid_url = "bolt://db.covidgraph.org:7687"
user = "public"
password = "corona"

#driver = GraphDatabase.driver(uri, auth=(user, password))
driver = GraphDatabase.driver(uri = covid_url,\
                              auth = (user,password))

##### Example of a paper node in the covid graph

In [3]:
paper_query = "MATCH (n:Paper) RETURN n LIMIT 1"
Data = []
with driver.session() as session:
    info = session.run(paper_query)
    for item in info:
        print(item)

<Record n=<Node id=2385529 labels=frozenset({'Paper'}) properties={'cord_uid': 'ocp6yodg', 'cord19-fulltext_hash': 'b8957d48b6bcf17b7b51e004d19314ce77f653a1', 'journal': 'BMC Infect Dis', 'publish_time': '2011-12-28', 'source': 'PMC', 'title': 'Timeliness of contact tracing among flight passengers for influenza A/H1N1 2009', '_hash_id': '84b069ab23fb0ecebe6925af9c2b18ae', 'url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3265549/'}>>


In [39]:
#query = "MATCH (g:GeneSymbol)<-[:MAPS]-(gn:Gene)<-[:ASSOCIATES]-(d:Disease) RETURN d LIMIT 10"
query = "MATCH (d:Disease)-->(gn:Gene) RETURN d LIMIT 1"

with driver.session() as session:
    info = session.run(query)
    for item in info:
        print(item)

<Record d=<Node id=3655483 labels=frozenset({'Disease'}) properties={'license': 'CC0 1.0', 'name': 'idiopathic pulmonary fibrosis', 'link': 'http://www.disease-ontology.org/?id=DOID:0050156', 'doid': 'DOID:0050156', 'definition': '"A pulmonary fibrosis that is characterized by scarring of the lung." [url:https\\://www.pulmonaryfibrosis.org/life-with-pf/about-ipf]', 'source': 'http://www.disease-ontology.org', '_id': 'e44b0dd76d1d22a1e9fe5c6c6dd395ab'}>>


In [43]:
#query = "MATCH (g:GeneSymbol)<-[:MAPS]-(gn:Gene)<-[:ASSOCIATES]-(d:Disease) RETURN d LIMIT 10"
query = "MATCH (d:Disease)-[rel:ASSOCIATES_DaG]->(gn:Gene) RETURN gn LIMIT 1"

with driver.session() as session:
    info = session.run(query)
    for item in info:
        print(item)

<Record gn=<Node id=95261 labels=frozenset({'Gene'}) properties={'Synonyms': 'MG1|MUC-5B|MUC5|MUC9', 'Symbol_from_nomenclature_authority': 'MUC5B', 'Symbol': 'MUC5B', 'chromosome': '11', 'description': 'mucin 5B, oligomeric mucus/gel-forming', 'Other_designations': 'mucin-5B|cervical mucin MUC5B|high molecular weight salivary mucin MG1|mucin 5, subtype B, tracheobronchial|sublingual gland mucin', 'source': 'ncbigene', 'type_of_gene': 'protein-coding', 'dbXrefs': 'MIM:600770|HGNC:HGNC:7516|Ensembl:ENSG00000117983', 'tax_id': '9606', 'sid': '727897', 'Nomenclature_status': 'O', 'map_location': '11p15.5', 'GeneID': '727897', 'Modification_date': '20200614', 'Feature_type': '-', 'LocusTag': '-', 'Full_name_from_nomenclature_authority': 'mucin 5B, oligomeric mucus/gel-forming'}>>


In [50]:
#query = "MATCH (g:GeneSymbol)<-[:MAPS]-(gn:Gene)<-[:ASSOCIATES]-(d:Disease) RETURN d LIMIT 10"
query = "MATCH (d:Disease)-[rel:ASSOCIATES_DaG]->(gn:Gene) \
         RETURN d.name,type(rel),gn.Symbol_from_nomenclature_authority LIMIT 5"

with driver.session() as session:
    info = session.run(query)
    for item in info:
        print(item)

<Record d.name='idiopathic pulmonary fibrosis' type(rel)='ASSOCIATES_DaG' gn.Symbol_from_nomenclature_authority='MUC5B'>
<Record d.name='idiopathic pulmonary fibrosis' type(rel)='ASSOCIATES_DaG' gn.Symbol_from_nomenclature_authority='TOLLIP'>
<Record d.name='idiopathic pulmonary fibrosis' type(rel)='ASSOCIATES_DaG' gn.Symbol_from_nomenclature_authority='MUC5AC'>
<Record d.name='idiopathic pulmonary fibrosis' type(rel)='ASSOCIATES_DaG' gn.Symbol_from_nomenclature_authority='TERT'>
<Record d.name='idiopathic pulmonary fibrosis' type(rel)='ASSOCIATES_DaG' gn.Symbol_from_nomenclature_authority='BRSK2'>


In [56]:
#query = "MATCH (g:GeneSymbol)<-[:MAPS]-(gn:Gene)<-[:ASSOCIATES]-(d:Disease) RETURN d LIMIT 10"
query = "MATCH (d:Disease)-[rel:ASSOCIATES_DaG]->(gn:Gene)-[mem:MEMBER]->(pw:Pathway) \
         RETURN d.name,gn.Symbol_from_nomenclature_authority, pw.name LIMIT 5"

with driver.session() as session:
    info = session.run(query)
    for item in info:
        print(item)

<Record d.name='idiopathic pulmonary fibrosis' gn.Symbol_from_nomenclature_authority='MUC5B' pw.name='Defective GALNT12 causes colorectal cancer 1 (CRCS1)'>
<Record d.name='idiopathic pulmonary fibrosis' gn.Symbol_from_nomenclature_authority='MUC5B' pw.name='O-linked glycosylation'>
<Record d.name='idiopathic pulmonary fibrosis' gn.Symbol_from_nomenclature_authority='MUC5B' pw.name='Post-translational protein modification'>
<Record d.name='idiopathic pulmonary fibrosis' gn.Symbol_from_nomenclature_authority='MUC5B' pw.name='Diseases associated with O-glycosylation of proteins'>
<Record d.name='idiopathic pulmonary fibrosis' gn.Symbol_from_nomenclature_authority='MUC5B' pw.name='Defective C1GALT1C1 causes Tn polyagglutination syndrome (TNPS)'>


In [64]:
#query = "MATCH (g:GeneSymbol)<-[:MAPS]-(gn:Gene)<-[:ASSOCIATES]-(d:Disease) RETURN d LIMIT 10"
query = "MATCH (pw:Pathway)<--(gn:Gene)-->(t:Transcript)-->(p:Protein) \
         RETURN pw.name,gn.synonyms,t.sid,p.sid LIMIT 5"

with driver.session() as session:
    info = session.run(query)
    for item in info:
        print(item)

<Record pw.name='Hemostasis' gn.synonyms=None t.sid='NM_001001579' p.sid='Q86WN5'>
<Record pw.name='Hemostasis' gn.synonyms=None t.sid='NM_001001579' p.sid='Q86SI6'>
<Record pw.name='Hemostasis' gn.synonyms=None t.sid='NM_001001579' p.sid='O76083'>
<Record pw.name='Hemostasis' gn.synonyms=None t.sid='NM_001001579' p.sid='O95225'>
<Record pw.name='Hemostasis' gn.synonyms=None t.sid='NM_001001579' p.sid='Q86SF7'>


In [65]:
#query = "MATCH (g:GeneSymbol)<-[:MAPS]-(gn:Gene)<-[:ASSOCIATES]-(d:Disease) RETURN d LIMIT 10"
query = "MATCH (d:Disease)-->(gn:Gene)-->(t:Transcript)-->(p:Protein) \
         RETURN d.name,gn.Symbol_from_nomenclature_authority,t.sid, p.sid LIMIT 5"

with driver.session() as session:
    info = session.run(query)
    for item in info:
        print(item)

<Record d.name='idiopathic pulmonary fibrosis' gn.Symbol_from_nomenclature_authority='MUC5B' t.sid='NM_002458' p.sid='O95451'>
<Record d.name='idiopathic pulmonary fibrosis' gn.Symbol_from_nomenclature_authority='MUC5B' t.sid='NM_002458' p.sid='Q9UE28'>
<Record d.name='idiopathic pulmonary fibrosis' gn.Symbol_from_nomenclature_authority='MUC5B' t.sid='NM_002458' p.sid='O00573'>
<Record d.name='idiopathic pulmonary fibrosis' gn.Symbol_from_nomenclature_authority='MUC5B' t.sid='NM_002458' p.sid='O00447'>
<Record d.name='idiopathic pulmonary fibrosis' gn.Symbol_from_nomenclature_authority='MUC5B' t.sid='NM_002458' p.sid='Q99552'>


### Counting

In [4]:
query = "MATCH (d:Disease)-->(gn:Gene)-->(t:Transcript)-->(p:Protein),\
        (gn:Gene)-->(pw:Pathway)\
         RETURN d.name, pw.name, count(gn), count(t), count(p) LIMIT 20"
Result = []
with driver.session() as session:
    info = session.run(query)
    for item in info:
            Result.append({"disease":item.values()[0],\
                     "pathway":item.values()[1],\
                     "g-count":item.values()[2],\
                    "t-count":item.values()[3],\
                    "p-count": item.values()[4]})
            
df = pd.DataFrame(Result)
df = df.set_index("disease")
df.head()

,g-count,p-count,pathway,t-count
disease,,,,
hypertension,43,43,2-LTR circle formation,43
attention deficit hyperactivity disorder,29,29,2-LTR circle formation,29
conduct disorder,29,29,2-LTR circle formation,29
endogenous depression,29,29,2-LTR circle formation,29
bipolar disorder,29,29,2-LTR circle formation,29


In [5]:
query = "MATCH (d:Disease)-->(gn:Gene)-->(t:Transcript)-->(p:Protein),\
        (pw:Pathway)<--(gn:Gene),\
        (d:Disease)-->(a:Anatomy)\
        RETURN d.name, pw.name,a.name, count(gn), count(t), count(p) LIMIT 20"

Result = []
with driver.session() as session:
    info = session.run(query)
    for item in info:
            Result.append({"disease":item.values()[0],\
                     "pathway":item.values()[1],\
                     "anatomy":item.values()[2],\
                     "g-count":item.values()[3],\
                    "t-count":item.values()[4],\
                    "p-count": item.values()[5]})
            
df = pd.DataFrame(Result)
#df = df.set_index("disease")
df.head()

,anatomy,disease,g-count,p-count,pathway,t-count
0,zona fasciculata of adrenal gland,hypertension,43,43,2-LTR circle formation,43
1,splenic artery,hypertension,43,43,2-LTR circle formation,43
2,anterior nucleus of hypothalamus,hypertension,43,43,2-LTR circle formation,43
3,common carotid artery plus branches,hypertension,43,43,2-LTR circle formation,43
4,forelimb zeugopod,hypertension,43,43,2-LTR circle formation,43


#### Count Pathways per Disease

In [6]:
query = "MATCH (d:Disease)-->(gn:Gene)-->(pw:Pathway)\
         RETURN d.name, pw.name, count(gn), count(pw) LIMIT 20"
Result = []
with driver.session() as session:
    info = session.run(query)
    for item in info:
            Result.append({"disease":item.values()[0],\
                     "pathway":item.values()[1],\
                     "g-count":item.values()[2],\
                    "pw-count": item.values()[3]})
            
df = pd.DataFrame(Result)
df = df.set_index("disease")
df.head()

,g-count,pathway,pw-count
disease,,,
hypertension,1,2-LTR circle formation,1
attention deficit hyperactivity disorder,1,2-LTR circle formation,1
conduct disorder,1,2-LTR circle formation,1
endogenous depression,1,2-LTR circle formation,1
bipolar disorder,1,2-LTR circle formation,1


In [14]:
query = "MATCH (d:Disease)-->(gn:Gene)-->(pw:Pathway)\
         RETURN d.name,collect(pw.name),count(pw.name) LIMIT 100"
Result = []
with driver.session() as session:
    info = session.run(query)
    for item in info:
            Result.append({"disease":item.values()[0],\
                    "pw-collect":item.values()[1],\
                    "pw-count": item.values()[2]})
            
df = pd.DataFrame(Result)
df = df.set_index("disease")
df.head(10)

,pw-collect,pw-count
disease,,
hypertension,"[2-LTR circle formation, ADORA2B mediated anti...",356
attention deficit hyperactivity disorder,"[2-LTR circle formation, A tetrasaccharide lin...",1215
conduct disorder,"[2-LTR circle formation, ADORA2B mediated anti...",218
endogenous depression,"[2-LTR circle formation, A tetrasaccharide lin...",1291
bipolar disorder,"[2-LTR circle formation, A tetrasaccharide lin...",1791
schizophrenia,"[2-LTR circle formation, A tetrasaccharide lin...",2989
lung cancer,[A tetrasaccharide linker sequence is required...,818
refractive error,[A tetrasaccharide linker sequence is required...,693
breast cancer,[A tetrasaccharide linker sequence is required...,1681


#### Count Proteins Per Pathways

In [18]:
query = "MATCH (pw:Pathway)<--(gn:Gene)-->(t:Transcript)-->(p:Protein)\
        RETURN pw.name,count(p.sid),collect(p.sid) LIMIT 100"


Result = []
with driver.session() as session:
    info = session.run(query)
    for item in info:
            Result.append({"pathway":item.values()[0],\
                     "p-count":item.values()[1],\
                     "p-collect":item.values()[2]})
            
df = pd.DataFrame(Result)
df = df.set_index("pathway")
df.head()

,p-collect,p-count
pathway,,
Hemostasis,"[Q86WN5, Q86SI6, O76083, O95225, Q86SF7, Q86SJ...",37326
Nitric oxide stimulates guanylate cyclase,"[Q86WN5, Q86SI6, O76083, O95225, Q86SF7, Q86SJ...",2613
Platelet homeostasis,"[Q86WN5, Q86SI6, O76083, O95225, Q86SF7, Q86SJ...",6859
cGMP effects,"[Q86WN5, Q86SI6, O76083, O95225, Q86SF7, Q86SJ...",2273
Developmental Biology,"[A9IZ92, Q96G25, A9IZ91, Q5JUY8, Q96FQ4, NP_00...",58720
